In [12]:
using RCall
@rimport stats as R
using Distributions

safediv(x, y) = x == 0 ? zero(x/y) : x/y

function pvalue_score(k, n, p)
    bin = Binomial(n, p)
    μ, σ = mean(bin), std(bin)
    z = safediv(k - μ, σ)
    2ccdf(Normal(), abs(z))
end

function confint_score(k, n; α = 0.05)
    p̂ = k/n
    c = quantile(Normal(), 1 - α/2)
    a, b, c = 1 + c^2/n, p̂ + c^2/(2n), p̂^2
    sqrtD = √(b^2 - a*c)
    [(b - sqrtD)/a, (b + sqrtD)/a]
end

function pvalue_confint_score(k, n; p=1/2, α=0.05)
    (pvalue=pvalue_score(k, n, p), confint=confint_score(k, n; α))
end

pvalue_confint_score (generic function with 1 method)

In [13]:
pvalue_confint_score(6, 20)

(pvalue = 0.07363827012030265, confint = [0.14547724486760408, 0.5189728183535237])

In [11]:
R.prop_test(6, 20; correct=false)

RObject{VecSxp}

	1-sample proportions test without continuity correction

data:  6L out of 20L, null probability 0.5
X-squared = 3.2, df = 1, p-value = 0.07364
alternative hypothesis: true p is not equal to 0.5
95 percent confidence interval:
 0.1454772 0.5189728
sample estimates:
  p 
0.3 



In [17]:
R.prop_test

RObject{ClosSxp}
function (x, n, p = NULL, alternative = c("two.sided", 
    "less", "greater"), conf.level = 0.95, correct = TRUE) 
{
    DNAME <- deparse1(substitute(x))
    if (is.table(x) && length(dim(x)) == 1L) {
        if (dim(x) != 2L) 
            stop("table 'x' should have 2 entries")
        l <- 1
        n <- sum(x)
        x <- x[1L]
    }
    else if (is.matrix(x)) {
        if (ncol(x) != 2L) 
            stop("'x' must have 2 columns")
        l <- nrow(x)
        n <- rowSums(x)
        x <- x[, 1L]
    }
    else {
        DNAME <- paste(DNAME, "out of", deparse1(substitute(n)))
        if ((l <- length(x)) != length(n)) 
            stop("'x' and 'n' must have the same length")
    }
    OK <- complete.cases(x, n)
    x <- x[OK]
    n <- n[OK]
    if ((k <- length(x)) < 1L) 
        stop("not enough data")
    if (any(n <= 0)) 
        stop("elements of 'n' must be positive")
    if (any(x < 0)) 
        stop("elements of 'x' must be nonnegative")
    if (any(x > 